<font color ='Navy'><center> __EMship+(Advanced Design of Sustainable
Ships and Offshore Structures)__ </center></font>
<br> Offshore Structures & Digital Twin <br>  __Structural health monitoring for offshore structures__ <br> 
***
***
<font color='Navy' size=6><b><center>Working with experimental data</center></b></font> 
***
    
<div class="alert alert-block alert-info">
<center><b>Don't forget: It is important that you run each cell of the notebook. To do so select a cell (it will be highlighted) and press shift+enter on your keyboard or the play button in the menu above.</b></center></div> 

In [ ]:
# let's import both numpy and matplotlib.pyplot 
# the "as" in the code bellow is to tell python that if we reference to the package using np and plt 
import numpy as np
import matplotlib.pyplot as plt 
# %matplotlib notebook

# 3. Operational Modal Analysis (OMA)

OMA is a **technique** used to identify the dynamic characteristics (natural frequencies, damping ratios, and mode shapes) of a structure **under its actual operating conditions**.  
Unlike traditional Experimental Modal Analysis (EMA), which requires **known excitation forces**, OMA relies only on **output measurements** (e.g., acceleration, displacement, or strain) recorded during ambient or operational loading.

OMA is particularly useful for **large and complex structures**—such as offshore wind turbines—where controlled excitation is impractical or impossible.  
In such cases, the ambient excitations (e.g., wind, waves, and machinery loads) are assumed to be **broadband and stochastic**, exciting the structure across a range of frequencies.

---



<img src="./Images/OMA.png" height=600 />

The general workflow of OMA involves:
1. **Data acquisition** – measuring responses (e.g., accelerations) at multiple points on the structure.  
2. **Preprocessing** – filtering, downsampling, and segmenting the data.  
3. **System identification** – estimating the modal parameters using algorithms such as:
   - Frequency Domain Decomposition (FDD)
   - Stochastic Subspace Identification (SSI)
   - Least-Squares Complex Frequency (LSCF)

4. **Modal validation** – evaluating stability diagrams and selecting consistent physical modes.

---

## OMA for Offshore Wind Turbines

In offshore wind turbines, OMA allows the extraction of:
- **Tower and blade natural frequencies** under operating wind and wave loads  
- **Damping ratios** influenced by aerodynamic and hydrodynamic effects  
- **Mode shapes** for structural health monitoring and model validation  

#### In this workshop, we will apply OMA algorithms using the **pyOMA2** package to measured acceleration signals.
---

First we will import the necessary packages.

In [ ]:
import pandas as pd
from pyoma2.algorithms import SSI, FSDD, pLSCF
from pyoma2.setup import SingleSetup

## 3.1 Parked condtion
 
Let us start with loading the parked condtion data

In [ ]:
# Read Parquet file with Pandas: 
file_path_parked = 'NRT-WTG_Parked.parquet.gz'  
data_parked = pd.read_parquet(file_path_parked)
data_parked

<div style="border-left: 1px solid black; padding: 1em; margin: 1em 0;">

<b> Assignment 3.1.1:</b> Store the 6 Acceleration signals in the varaible `parked_signals`. Also store the sampling frequncy of the signals in `parked_fs`.

</div>

In [ ]:
acc_columns = [col for col in data_parked.columns if 'ACC' in col]
parked_signals = data_parked[acc_columns] # Your ACC DataFrame
parked_fs = 30 # ACC sampling frequency
parked_signals = parked_signals.replace("Missing value", float("nan"))
parked_signals = parked_signals.dropna(how='all')
parked_signals =  parked_signals - parked_signals.mean() # Detrend by removing mean
parked_signals

Let us now look at the accelration signals and some statistics

In [ ]:
plt.figure(figsize=(9,5))
for c in parked_signals.columns:
    plt.plot(parked_signals.index, parked_signals[c], label=c)
plt.legend()
plt.ylabel('Acceleration (g)')
plt.show()

In [ ]:
stats = pd.DataFrame({
    "Mean": parked_signals.mean(),
    "Std": parked_signals.std(),
    "RMS": np.sqrt((parked_signals**2).mean()),
    "Min": parked_signals.min(),
    "Max": parked_signals.max()
})

stats.plot(kind="bar", figsize=(14,4), grid=True, alpha=0.8)
plt.title("Signal Statistics")
plt.ylabel("Value")
plt.xticks(rotation=45)
plt.show()

<div style="border-left: 1px solid black; padding: 1em; margin: 1em 0;">

<b> Assignment 3.1.2:</b> Store the SCADA data (e.g. wind direction, yaw angle, rpm, power, wind speed, etc) in the varaible `SCADA_signals`. Also store the sampling frequncy of the signals in `SCADA_fs`.

</div>

In [ ]:
# Select SCADA column names by keywords
scada_keywords = ["winddirection [°]", "rotorspeed [rpm]", "pitch [°]", "power [kW]", "yaw [°]", "windspeed [m/s]"]
SCADA_columns = [col for col in data_parked.columns if any(key.upper() in col.upper() for key in scada_keywords)]
SCADA_signals_p = data_parked[SCADA_columns]
SCADA_signals_p = SCADA_signals_p.replace("Missing value", float("nan"))
SCADA_signals_p = SCADA_signals_p.dropna(how='all')
SCADA_fs = 1 # SCADA sampling frequency in Hz
SCADA_signals_p

In [ ]:
fig, ax = plt.subplots(figsize=(9,5))
for c in SCADA_signals_p.columns:
    ax.plot(SCADA_signals_p.index, SCADA_signals_p[c], label=c) 
plt.legend()
plt.ylabel('SCADA Signals')
plt.show()

<div style="border-left: 1px solid black; padding: 1em; margin: 1em 0;">

<b> Assignment 3.1.3:</b> Rotate the accelrations data to Fore-Aft (FA) and Side-Side (SS) refernce frame of the turbine. Use the function `rotate_sensor_to_fa_ss` and store your FA and SS Accelration signals in `FA_ACC_LAT15` etc..

</div>

In [ ]:
def rotate_sensor_to_fa_ss(x_s, y_s, yaw_deg, sensor_loc_deg):
    
    sensor_x_bearing = (sensor_loc_deg + 180) % 360 # Sensor X-axis points inward → add 180°
    yaw_deg = yaw_deg % 360 

    delta = ((yaw_deg - sensor_x_bearing + 180) % 360) - 180 # Rotation angle in [-180, 180]
    d = np.deg2rad(delta)

    FA =  np.cos(d) * x_s + np.sin(d) * y_s
    SS = -np.sin(d) * x_s + np.cos(d) * y_s
    return FA, SS


In [ ]:
FA_ACC_LAT15, SS_ACC_LAT15 = None, None
FA_ACC_LAT69, SS_ACC_LAT69 = None, None
FA_ACC_LAT97, SS_ACC_LAT97 = None, None

yaw_30Hz = np.repeat(SCADA_signals_p['yaw [°]'].to_numpy(), 30) # enshure yaw is at 30Hz with Acc data

### BEGIN SOLUTION
FA_ACC_LAT15, SS_ACC_LAT15 = rotate_sensor_to_fa_ss(
    x_s=parked_signals['TP_ACC_LAT015_DEG240_X_nr1 [g]'].to_numpy(),
    y_s=parked_signals['TP_ACC_LAT015_DEG240_Y_nr2 [g]'].to_numpy(),
    yaw_deg=yaw_30Hz,
    sensor_loc_deg=240)

FA_ACC_LAT69, SS_ACC_LAT69= rotate_sensor_to_fa_ss(
    x_s=parked_signals['TW_ACC_LAT069_DEG060_X_nr1 [g]'].to_numpy(),
    y_s=parked_signals['TW_ACC_LAT069_DEG060_Y_nr2 [g]'].to_numpy(),
    yaw_deg=yaw_30Hz,
    sensor_loc_deg=60)

FA_ACC_LAT97, SS_ACC_LAT97 = rotate_sensor_to_fa_ss(
    x_s=parked_signals['TW_ACC_LAT097_DEG060_X_nr1 [g]'].to_numpy(),
    y_s=parked_signals['TW_ACC_LAT097_DEG060_Y_nr2 [g]'].to_numpy(),
    yaw_deg=yaw_30Hz,
    sensor_loc_deg=60)
### END SOLUTION

Let us visualize your signals after transformation

In [ ]:
fig, ax = plt.subplots(3,2, figsize=(16,12))
ax[0,0].plot(FA_ACC_LAT97, label='FA LAT97')
ax[0,0].plot(parked_signals['TW_ACC_LAT097_DEG060_X_nr1 [g]'].to_numpy(), label='Original X LAT97', linestyle='--')
ax[0,0].legend()
ax[0,0].set_title('LAT97 Sensor') 

ax[1,0].plot(FA_ACC_LAT69, label='FA LAT69')
ax[1,0].plot(parked_signals['TW_ACC_LAT069_DEG060_X_nr1 [g]'].to_numpy(), label='Original X LAT69', linestyle='--')
ax[1,0].legend()
ax[1,0].set_title('LAT69 Sensor') 

ax[2,0].plot(FA_ACC_LAT15, label='FA LAT15')
ax[2,0].plot(parked_signals['TP_ACC_LAT015_DEG240_X_nr1 [g]'].to_numpy(), label='Original X LAT15', linestyle='--')
ax[2,0].legend()
ax[2,0].set_title('LAT15 Sensor') 

ax[0,1].plot(SS_ACC_LAT97, label='SS LAT97', color='orange')
ax[0,1].plot(parked_signals['TW_ACC_LAT097_DEG060_Y_nr2 [g]'].to_numpy(), label='Original Y LAT97', linestyle='--', color='red')
ax[0,1].legend()
ax[0,1].set_title('LAT97 Sensor')   

ax[1,1].plot(SS_ACC_LAT69, label='SS LAT69', color='orange')
ax[1,1].plot(parked_signals['TW_ACC_LAT069_DEG060_Y_nr2 [g]'].to_numpy(), label='Original Y LAT69', linestyle='--', color='red')
ax[1,1].legend()
ax[1,1].set_title('LAT69 Sensor')   

ax[2,1].plot(SS_ACC_LAT15, label='SS LAT15', color='orange')
ax[2,1].plot(parked_signals['TP_ACC_LAT015_DEG240_Y_nr2 [g]'].to_numpy(), label='Original Y LAT15', linestyle='--', color='red')
ax[2,1].legend()
ax[2,1].set_title('LAT15 Sensor')
plt.show()


And now let us look at the PSDs 

In [ ]:
from scipy.signal import welch
from scipy.signal.windows import hann

f_X_97, X_97 = welch(parked_signals['TW_ACC_LAT097_DEG060_X_nr1 [g]'].to_numpy(), fs=parked_fs, window=hann(parked_fs*600))
f_Y_97, Y_97 = welch(parked_signals['TW_ACC_LAT097_DEG060_Y_nr2 [g]'].to_numpy(), fs=parked_fs, window=hann(parked_fs*600))
f_X_69, X_69 = welch(parked_signals['TW_ACC_LAT069_DEG060_X_nr1 [g]'].to_numpy(), fs=parked_fs, window=hann(parked_fs*600))
f_Y_69, Y_69 = welch(parked_signals['TW_ACC_LAT069_DEG060_Y_nr2 [g]'].to_numpy(), fs=parked_fs, window=hann(parked_fs*600))
f_X_15, X_15 = welch(parked_signals['TP_ACC_LAT015_DEG240_X_nr1 [g]'].to_numpy(), fs=parked_fs, window=hann(parked_fs*600))
f_Y_15, Y_15 = welch(parked_signals['TP_ACC_LAT015_DEG240_Y_nr2 [g]'].to_numpy(), fs=parked_fs, window=hann(parked_fs*600))

f_psd_FA_LAT97_parked, psd_FA_LAT97_parked = welch(FA_ACC_LAT97, fs=parked_fs, window=hann(parked_fs*600))
f_psd_SS_LAT97_parked, psd_SS_LAT97_parked = welch(SS_ACC_LAT97, fs=parked_fs, window=hann(parked_fs*600))
f_psd_FA_LAT69_parked, psd_FA_LAT69_parked = welch(FA_ACC_LAT69, fs=parked_fs, window=hann(parked_fs*600))
f_psd_SS_LAT69_parked, psd_SS_LAT69_parked = welch(SS_ACC_LAT69, fs=parked_fs, window=hann(parked_fs*600))
f_psd_FA_LAT15_parked, psd_FA_LAT15_parked = welch(FA_ACC_LAT15, fs=parked_fs, window=hann(parked_fs*600))
f_psd_SS_LAT15_parked, psd_SS_LAT15_parked = welch(SS_ACC_LAT15, fs=parked_fs, window=hann(parked_fs*600))

psd_y_label = 'g²/Hz'


In [ ]:
fig, ax = plt.subplots(3,2, figsize=(16,12))
ax[0,0].semilogy(f_psd_FA_LAT97_parked, psd_FA_LAT97_parked, label='FA LAT97')
ax[0,0].semilogy(f_X_97, X_97, label='Original X LAT97', linestyle='--')
ax[0,0].set_ylabel(psd_y_label)
ax[0,0].set_title('FA LAT97 Sensor PSD')
ax[0,0].set_xlim(0, 2)
ax[0,0].legend()    

ax[1,0].semilogy(f_psd_FA_LAT69_parked, psd_FA_LAT69_parked, label='FA LAT69')
ax[1,0].semilogy(f_X_69, X_69, label='Original X LAT69', linestyle='--')
ax[1,0].set_ylabel(psd_y_label)
ax[1,0].set_title('FA LAT69 Sensor PSD')
ax[1,0].legend()
ax[1,0].set_xlim(0, 2)

ax[2,0].semilogy(f_psd_FA_LAT15_parked, psd_FA_LAT15_parked, label='FA LAT15')
ax[2,0].semilogy(f_X_15, X_15, label='Original X LAT15', linestyle='--')
ax[2,0].set_ylabel(psd_y_label)
ax[2,0].set_title('FA LAT15 Sensor PSD')
ax[2,0].set_xlabel('Frequency (Hz)')
ax[2,0].legend()
ax[2,0].set_xlim(0, 2)

ax[0,1].semilogy(f_psd_SS_LAT97_parked, psd_SS_LAT97_parked, label='SS LAT97', color='orange')
ax[0,1].semilogy(f_Y_97, Y_97, label='Original Y LAT97', linestyle='--', color='red')
ax[0,1].set_ylabel(psd_y_label)
ax[0,1].set_title('SS LAT97 Sensor PSD')
ax[0,1].legend()
ax[0,1].set_xlim(0, 2)

ax[1,1].semilogy(f_psd_SS_LAT69_parked, psd_SS_LAT69_parked, label='SS LAT69', color='orange')
ax[1,1].semilogy(f_Y_69, Y_69, label='Original Y LAT69', linestyle='--', color='red')
ax[1,1].set_ylabel(psd_y_label)
ax[1,1].set_title('SS LAT69 Sensor PSD')
ax[1,1].legend()
ax[1,1].set_xlim(0, 2)

ax[2,1].semilogy(f_psd_SS_LAT15_parked, psd_SS_LAT15_parked, label='SS LAT15', color='orange')
ax[2,1].semilogy(f_Y_15, Y_15, label='Original Y LAT15', linestyle='--', color='red')
ax[2,1].set_ylabel(psd_y_label)
ax[2,1].set_title('SS LAT15 Sensor PSD')
ax[2,1].set_xlabel('Frequency (Hz)')
ax[2,1].legend()
ax[2,1].set_xlim(0, 2)

<div style="border-left: 1px solid black; padding: 1em; margin: 1em 0;">

<b> Assignment 3.1.4:</b> Store the FA and SS signals in one varaiable called `OMA_parked_signals` so we can pass this varaible to pyOMA2 and perform Operational Modal Analysis. Let us take only 600s of the signals.

</div>

In [ ]:
OMA_parked_signals = None
### BEGIN SOLUTION
OMA_parked_signals = np.column_stack((
    FA_ACC_LAT15[600*30:1200*30],
    SS_ACC_LAT15[600*30:1200*30],
    FA_ACC_LAT69[600*30:1200*30],
    SS_ACC_LAT69[600*30:1200*30],
    FA_ACC_LAT97[600*30:1200*30],
    SS_ACC_LAT97[600*30:1200*30]
))
### END SOLUTION
signals_p = SingleSetup(OMA_parked_signals, fs=parked_fs)
df_n = ['FA_ACC_LAT15 [g]', 'SS_ACC_LAT15 [g]','FA_ACC_LAT69 [g]', 'SS_ACC_LAT69 [g]','FA_ACC_LAT97 [g]', 'SS_ACC_LAT97 [g]']
signals_p.plot_data(names=df_n, unit='g')
plt.show()

Let us see more inforamtion generated by pyOMA2 for the first signal (LAT015_FA)

In [ ]:
signals_p.plot_ch_info(ch_idx=[0])
plt.show()

<div style="border-left: 1px solid black; padding: 1em; margin: 1em 0;">

<b> Assignment 3.1.5:</b> Use the `FSDD` function applied to parked signals in `signals_p`. Store you results in `FSDD_results_parked`.

**Tip**: Use `FSDD(name="FSDD", nxseg=1024)`

</div>

In [ ]:
FSDD_results_parked = None
### BEGIN SOLUTION
FSDD_results_parked = FSDD(name="FSDD", nxseg=1024*2)
### END SOLUTION
signals_p.add_algorithms(FSDD_results_parked)
signals_p.run_by_name("FSDD")
_, _ = FSDD_results_parked.plot_CMIF(freqlim=(0,2))
plt.show()

<div style="border-left: 1px solid black; padding: 1em; margin: 1em 0;">

<b> Assignment 3.1.6:</b> Use the `SSI` function applied to `signals`. Store you results in `SSI_results`.

**Tip**: Use `SSI(name="SSIcov", method="cov", br=60, ordmax=60)`

</div>

In [ ]:
SSI_results_parked = None
### BEGIN SOLUTION
SSI_results_parked = SSI(name="SSIcov", method="cov", br=60, ordmax=60)
### END SOLUTION
signals_p.add_algorithms(SSI_results_parked)
signals_p.run_by_name("SSIcov")
SSI_results_parked.plot_stab(freqlim=(0,2), hide_poles=False, spectrum=True)
SSI_results_parked.plot_freqvsdamp(freqlim=(0,2))
plt.show()

### Finally, let us see the mode shapes of the SSI and LSCF algorithms.

In [ ]:
from mode_shapes import plot_static_mode_shape

SSI_mode_shapes = SSI_results_parked.result.Phi_poles
SSI_freqs = SSI_results_parked.result.Fn_poles

plot_static_mode_shape(target_freq=0.23, tol=0.1, ssi_order=40, SSI_mode_shapes=SSI_mode_shapes, SSI_freqs=SSI_freqs)
plot_static_mode_shape(target_freq=1.3, tol=0.1, ssi_order=10, SSI_mode_shapes=SSI_mode_shapes, SSI_freqs=SSI_freqs)

## 3.2 Operation (rated) condtion
 
Let us start with loading the rated condtion data

In [ ]:
file_path_rated = 'NRT-WTG_Rated.parquet.gz' 
data_rated = pd.read_parquet(file_path_rated) 
data_rated

<div style="border-left: 1px solid black; padding: 1em; margin: 1em 0;">

<b> Assignment 3.2.1:</b> Store the 6 Acceleration signals in the varaible `rated_signals`. Also store the sampling frequncy of the signals in `rated_fs`.

</div>

In [ ]:
acc_columns = [col for col in data_rated.columns if 'ACC' in col]
rated_signals = data_rated[acc_columns] # Your ACC DataFrame
rated_fs = 30 # ACC sampling frequency
rated_signals = rated_signals.replace("Missing value", float("nan"))
rated_signals = rated_signals.dropna(how='all')
rated_signals =  rated_signals - rated_signals.mean() # Detrend by removing mean
rated_signals

Let us look into the signals and some statstics

In [ ]:
plt.figure(figsize=(9,5))
for c in rated_signals.columns:
    plt.plot(rated_signals.index, rated_signals[c], label=c)
plt.legend()
plt.ylabel('Acceleration (g)')
plt.show()

In [ ]:
stats_r = pd.DataFrame({
    "Mean": rated_signals.mean(),
    "Std": rated_signals.std(),
    "RMS": np.sqrt((rated_signals**2).mean()),
    "Min": rated_signals.min(),
    "Max": rated_signals.max()
})

stats_r.plot(kind="bar", figsize=(14,4), grid=True, alpha=0.8)
plt.title("Signal Statistics")
plt.ylabel("Value")
plt.xticks(rotation=45)
plt.show()

<div style="border-left: 1px solid black; padding: 1em; margin: 1em 0;">

<b> Assignment 3.2.2:</b> Store the SCADA data (e.g. wind direction, yaw angle, rpm, power, wind speed, etc) in the varaible `SCADA_signals`. Also store the sampling frequncy of the signals in `SCADA_fs`.

</div>

In [ ]:
# Select SCADA column names by keywords
scada_keywords = ["winddirection [°]", "rotorspeed [rpm]", "pitch [°]", "power [kW]", "yaw [°]", "windspeed [m/s]"]
SCADA_columns = [col for col in data_rated.columns if any(key.upper() in col.upper() for key in scada_keywords)]
SCADA_signals_rated = data_rated[SCADA_columns]
SCADA_signals_rated = SCADA_signals_rated.replace("Missing value", float("nan"))
SCADA_signals_rated = SCADA_signals_rated.dropna(how='all')
SCADA_fs = 1 # SCADA sampling frequency in Hz
SCADA_signals_rated

In [ ]:
plt.figure(figsize=(8,2))
plt.plot(SCADA_signals_rated.index, SCADA_signals_rated['power [kW]'], label='power [kW]')
plt.legend()

plt.figure(figsize=(8,2))
plt.plot(SCADA_signals_rated.index, SCADA_signals_rated['windspeed [m/s]'], label='windspeed [m/s]', color='orange')
plt.legend()

plt.figure(figsize=(8,2))
plt.plot(SCADA_signals_rated.index, SCADA_signals_rated['winddirection [°]'], label='wind direction [°]', color='red', linestyle='--')
plt.plot(SCADA_signals_rated.index, SCADA_signals_rated['yaw [°]'], label='yaw [°]', color='blue')
plt.legend()

plt.figure(figsize=(8,2))
plt.plot(SCADA_signals_rated.index, SCADA_signals_rated['pitch [°]'], label='pitch [°]', color='green')
plt.plot(SCADA_signals_rated.index, SCADA_signals_rated['rotorspeed [rpm]'], label='rotorspeed [rpm]', color='orange')
plt.legend()

<div style="border-left: 1px solid black; padding: 1em; margin: 1em 0;">

<b> Assignment 3.2.3:</b> Rotate the accelrations data to Fore-Aft (FA) and Side-Side (SS) refernce frame of the turbine. Use the function `rotate_sensor_to_fa_ss` and store your FA and SS Accelration signals in `FA_ACC_LAT15` etc..

</div>

In [ ]:
FA_ACC_LAT15_rated, SS_ACC_LAT15_rated = None, None
FA_ACC_LAT69_rated, SS_ACC_LAT69_rated = None, None
FA_ACC_LAT97_rated, SS_ACC_LAT97_rated = None, None

yaw_30Hz = np.repeat(SCADA_signals_rated['yaw [°]'].to_numpy(), 30) # enshure yaw is at 30Hz with Acc data

### BEGIN SOLUTION
FA_ACC_LAT15_rated, SS_ACC_LAT15_rated = rotate_sensor_to_fa_ss(
    x_s=rated_signals['TP_ACC_LAT015_DEG240_X_nr1 [g]'].to_numpy(),
    y_s=rated_signals['TP_ACC_LAT015_DEG240_Y_nr2 [g]'].to_numpy(),
    yaw_deg=yaw_30Hz,
    sensor_loc_deg=240)

FA_ACC_LAT69_rated, SS_ACC_LAT69_rated= rotate_sensor_to_fa_ss(
    x_s=rated_signals['TW_ACC_LAT069_DEG060_X_nr1 [g]'].to_numpy(),
    y_s=rated_signals['TW_ACC_LAT069_DEG060_Y_nr2 [g]'].to_numpy(),
    yaw_deg=yaw_30Hz,
    sensor_loc_deg=60)

FA_ACC_LAT97_rated, SS_ACC_LAT97_rated = rotate_sensor_to_fa_ss(
    x_s=rated_signals['TW_ACC_LAT097_DEG060_X_nr1 [g]'].to_numpy(),
    y_s=rated_signals['TW_ACC_LAT097_DEG060_Y_nr2 [g]'].to_numpy(),
    yaw_deg=yaw_30Hz,
    sensor_loc_deg=60)
### END SOLUTION

Let us visualize your signals after transformation

In [ ]:
fig, ax = plt.subplots(3,2, figsize=(16,12))
ax[0,0].plot(FA_ACC_LAT97_rated, label='FA LAT97')
ax[0,0].plot(rated_signals['TW_ACC_LAT097_DEG060_X_nr1 [g]'].to_numpy(), label='Original X LAT97', linestyle='--')
    
ax[0,0].set_title('LAT97 Sensor') 

ax[1,0].plot(FA_ACC_LAT69_rated, label='FA LAT69')
ax[1,0].plot(rated_signals['TW_ACC_LAT069_DEG060_X_nr1 [g]'].to_numpy(), label='Original X LAT69', linestyle='--')
ax[1,0].set_title('LAT69 Sensor') 

ax[2,0].plot(FA_ACC_LAT15_rated, label='FA LAT15')
ax[2,0].plot(rated_signals['TP_ACC_LAT015_DEG240_X_nr1 [g]'].to_numpy(), label='Original X LAT15', linestyle='--')
ax[2,0].set_title('LAT15 Sensor') 

ax[0,1].plot(SS_ACC_LAT97_rated, label='SS LAT97', color='orange')
ax[0,1].plot(rated_signals['TW_ACC_LAT097_DEG060_Y_nr2 [g]'].to_numpy(), label='Original Y LAT97', linestyle='--', color='red')
ax[0,1].set_title('LAT97 Sensor')   

ax[1,1].plot(SS_ACC_LAT69_rated, label='SS LAT69', color='orange')
ax[1,1].plot(rated_signals['TW_ACC_LAT069_DEG060_Y_nr2 [g]'].to_numpy(), label='Original Y LAT69', linestyle='--', color='red')
ax[1,1].set_title('LAT69 Sensor')   
ax[2,1].plot(SS_ACC_LAT15_rated, label='SS LAT15', color='orange')
ax[2,1].plot(rated_signals['TP_ACC_LAT015_DEG240_Y_nr2 [g]'].to_numpy(), label='Original Y LAT15', linestyle='--', color='red')
ax[2,1].set_title('LAT15 Sensor')



PSD

In [ ]:
from scipy.signal import welch
from scipy.signal.windows import hann

f_X_97_r, X_97_r = welch(rated_signals['TW_ACC_LAT097_DEG060_X_nr1 [g]'].to_numpy(), fs=rated_fs, window=hann(rated_fs*600))
f_Y_97_r, Y_97_r = welch(rated_signals['TW_ACC_LAT097_DEG060_Y_nr2 [g]'].to_numpy(), fs=rated_fs, window=hann(rated_fs*600))
f_X_69_r, X_69_r = welch(rated_signals['TW_ACC_LAT069_DEG060_X_nr1 [g]'].to_numpy(), fs=rated_fs, window=hann(rated_fs*600))
f_Y_69_r, Y_69_r = welch(rated_signals['TW_ACC_LAT069_DEG060_Y_nr2 [g]'].to_numpy(), fs=rated_fs, window=hann(rated_fs*600))
f_X_15_r, X_15_r = welch(rated_signals['TP_ACC_LAT015_DEG240_X_nr1 [g]'].to_numpy(), fs=rated_fs, window=hann(rated_fs*600))
f_Y_15_r, Y_15_r = welch(rated_signals['TP_ACC_LAT015_DEG240_Y_nr2 [g]'].to_numpy(), fs=rated_fs, window=hann(rated_fs*600))

f_psd_FA_LAT97_rated, psd_FA_LAT97_rated = welch(FA_ACC_LAT97, fs=rated_fs, window=hann(rated_fs*600))
f_psd_SS_LAT97_rated, psd_SS_LAT97_rated = welch(SS_ACC_LAT97, fs=rated_fs, window=hann(rated_fs*600))
f_psd_FA_LAT69_rated, psd_FA_LAT69_rated = welch(FA_ACC_LAT69, fs=rated_fs, window=hann(rated_fs*600))
f_psd_SS_LAT69_rated, psd_SS_LAT69_rated = welch(SS_ACC_LAT69, fs=rated_fs, window=hann(rated_fs*600))
f_psd_FA_LAT15_rated, psd_FA_LAT15_rated = welch(FA_ACC_LAT15, fs=rated_fs, window=hann(rated_fs*600))
f_psd_SS_LAT15_rated, psd_SS_LAT15_rated = welch(SS_ACC_LAT15, fs=rated_fs, window=hann(rated_fs*600))

psd_y_label = 'g²/Hz'

In [ ]:
fig, ax = plt.subplots(3,2, figsize=(16,12))
ax[0,0].semilogy(f_psd_FA_LAT97_rated, psd_FA_LAT97_rated, label='FA LAT97')
ax[0,0].semilogy(f_X_97_r, X_97_r, label='Original X LAT97', linestyle='--')
ax[0,0].set_ylabel(psd_y_label)
ax[0,0].set_title('FA LAT97 Sensor PSD')
ax[0,0].set_xlim(0, 2)
ax[0,0].legend()    

ax[1,0].semilogy(f_psd_FA_LAT69_rated, psd_FA_LAT69_rated, label='FA LAT69')
ax[1,0].semilogy(f_X_69_r, X_69_r, label='Original X LAT69', linestyle='--')
ax[1,0].set_ylabel(psd_y_label)
ax[1,0].set_title('FA LAT69 Sensor PSD')
ax[1,0].legend()
ax[1,0].set_xlim(0, 2)

ax[2,0].semilogy(f_psd_FA_LAT15_rated, psd_FA_LAT15_rated, label='FA LAT15')
ax[2,0].semilogy(f_X_15_r, X_15_r, label='Original X LAT15', linestyle='--')
ax[2,0].set_ylabel(psd_y_label)
ax[2,0].set_title('FA LAT15 Sensor PSD')
ax[2,0].set_xlabel('Frequency (Hz)')
ax[2,0].legend()
ax[2,0].set_xlim(0, 2)

ax[0,1].semilogy(f_psd_SS_LAT97_rated, psd_SS_LAT97_rated, label='SS LAT97', color='orange')
ax[0,1].semilogy(f_Y_97_r, Y_97_r, label='Original Y LAT97', linestyle='--', color='red')
ax[0,1].set_ylabel(psd_y_label)
ax[0,1].set_title('SS LAT97 Sensor PSD')
ax[0,1].legend()
ax[0,1].set_xlim(0, 2)

ax[1,1].semilogy(f_psd_SS_LAT69_rated, psd_SS_LAT69_rated, label='SS LAT69', color='orange')
ax[1,1].semilogy(f_Y_69_r, Y_69_r, label='Original Y LAT69', linestyle='--', color='red')
ax[1,1].set_ylabel(psd_y_label)
ax[1,1].set_title('SS LAT69 Sensor PSD')
ax[1,1].legend()
ax[1,1].set_xlim(0, 2)

ax[2,1].semilogy(f_psd_SS_LAT15_rated, psd_SS_LAT15_rated, label='SS LAT15', color='orange')
ax[2,1].semilogy(f_Y_15_r, Y_15_r, label='Original Y LAT15', linestyle='--', color='red')
ax[2,1].set_ylabel(psd_y_label)
ax[2,1].set_title('SS LAT15 Sensor PSD')
ax[2,1].set_xlabel('Frequency (Hz)')
ax[2,1].legend()
ax[2,1].set_xlim(0, 2)

<div style="border-left: 1px solid black; padding: 1em; margin: 1em 0;">

<b> Assignment 3.2.4:</b> Store the FA and SS signals in one varaiable called `OMA_rated_signals` so we can pass this varaible to pyOMA2 and perform Operational Modal Analysis. Let us take only 600s of the signals.

</div>

In [ ]:
OMA_rated_signals = None
### BEGIN SOLUTION
OMA_rated_signals = np.column_stack((
    FA_ACC_LAT15_rated[600*30:1200*30],
    SS_ACC_LAT15_rated[600*30:1200*30],
    FA_ACC_LAT69_rated[600*30:1200*30],
    SS_ACC_LAT69_rated[600*30:1200*30],
    FA_ACC_LAT97_rated[600*30:1200*30],
    SS_ACC_LAT97_rated[600*30:1200*30]
))
### END SOLUTION
signals_r = SingleSetup(OMA_rated_signals, fs=rated_fs)
df_n = ['FA_ACC_LAT15 [g]', 'SS_ACC_LAT15 [g]','FA_ACC_LAT69 [g]', 'SS_ACC_LAT69 [g]','FA_ACC_LAT97 [g]', 'SS_ACC_LAT97 [g]']
signals_r.plot_data(names=df_n, unit='g')
plt.show()

Let us see more inforamtion generated by pyOMA2 for the first signal (LAT015_FA)

In [ ]:
signals_r.plot_ch_info(ch_idx=[0])
plt.show()

<div style="border-left: 1px solid black; padding: 1em; margin: 1em 0;">

<b> Assignment 3.2.5:</b> Use the `FSDD` function applied to parked signals in `signals_r`. Store you results in `FSDD_results_rated`.

**Tip**: Use `FSDD(name="FSDD", nxseg=1024)`

</div>

In [ ]:
FSDD_results_rated = None
### BEGIN SOLUTION
FSDD_results_rated = FSDD(name="FSDD", nxseg=1024*2)
### END SOLUTION
signals_r.add_algorithms(FSDD_results_rated)
signals_r.run_by_name("FSDD")
_, _ = FSDD_results_rated.plot_CMIF(freqlim=(0,2))
plt.show()

<div style="border-left: 1px solid black; padding: 1em; margin: 1em 0;">

<b> Assignment 3.2.6:</b> Use the `SSI` function applied to `signals_r`. Store you results in `SSI_results_rated`.

**Tip**: Use `SSI(name="SSIcov", method="cov", br=60, ordmax=60)`

</div>

In [ ]:
SSI_results_rated = None
### BEGIN SOLUTION
SSI_results_rated = SSI(name="SSIcov", method="cov", br=80, ordmax=60)
### END SOLUTION
signals_r.add_algorithms(SSI_results_rated)
signals_r.run_by_name("SSIcov")
SSI_results_rated.plot_stab(freqlim=(0,2), hide_poles=False, spectrum=True)
SSI_results_rated.plot_freqvsdamp(freqlim=(0,2))
plt.show()

### Finally, let us see the mode shapes of the SSI and LSCF algorithms.

In [ ]:
from mode_shapes import plot_static_mode_shape

SSI_mode_shapes = SSI_results_rated.result.Phi_poles
SSI_freqs = SSI_results_rated.result.Fn_poles

plot_static_mode_shape(target_freq=0.25, tol=0.2, ssi_order=10, SSI_mode_shapes=SSI_mode_shapes, SSI_freqs=SSI_freqs)
plot_static_mode_shape(target_freq=1.3, tol=0.1, ssi_order=10, SSI_mode_shapes=SSI_mode_shapes, SSI_freqs=SSI_freqs)

<div class="alert alert-block alert-info">
<center><b>This concludes this workshop, hope you learned a thing or two..</b></center></div> 